In [1]:
from pathlib import Path

import numpy as np
import polars as pl
from tqdm.auto import tqdm

/opt/mamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "../data"
N_ROWS = 500000000
files = list(Path(data_dir).glob("raw_train*"))
print(f"# of train files: {len(files)}")

# of train files: 100


In [3]:
weight_df = pl.read_csv(Path(data_dir, "sample_submission.csv"), n_rows=1)
label_cols = weight_df.columns[1:]
tmp_df = pl.read_csv(Path(data_dir, "train.csv"), n_rows=1)
feat_cols = tmp_df.select(pl.exclude(label_cols)).columns[1:]
len(feat_cols), len(label_cols)

(556, 368)

In [4]:
def calculate_statistics(cols, files, chunk_size=50):
    stats = []
    gs = (len(cols) - 1) // chunk_size + 1
    for i in tqdm(range(gs)):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        sub_cols = cols[start: end]
        dfs = []
        num_data = 0
        for filename in files:
            dfs.append(pl.read_parquet(filename, columns=sub_cols))
            num_data += len(dfs[-1])
            if num_data > N_ROWS:
                break
        df = pl.concat(dfs)
        for col in sub_cols:
            m = df.filter(pl.col(col) > 1e-7)[col].mean()
            m = m if m is not None else 1e-50
            lambda_val = 1 / m
            stats.append({
                "mean": df[col].mean(),
                "std": df[col].std(),
                "q1_4": df[col].quantile(0.25),
                "q2_4": df[col].quantile(0.5),
                "q3_4": df[col].quantile(0.75),
                "min": df[col].min(),
                "max": df[col].max(),
                "lambda": lambda_val,
                "std_y": np.sqrt((df[col]**2).mean()),
            })
    stats_df = (
        pl.from_dicts(stats)
        .transpose(include_header=True, header_name="stats", column_names=cols)
    )
    return stats_df


feat_stats_df = calculate_statistics(feat_cols, files)
label_stats_df = calculate_statistics(label_cols, files)

100% 12/12 [09:40<00:00, 48.41s/it]
100% 8/8 [06:00<00:00, 45.08s/it]


In [5]:
feat_stats_df.shape, label_stats_df.shape

((9, 557), (9, 369))

In [6]:
stats_df = pl.concat([feat_stats_df, label_stats_df[:,1:]], how="horizontal")
stats_df

stats,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,state_t_9,state_t_10,state_t_11,state_t_12,state_t_13,state_t_14,state_t_15,state_t_16,state_t_17,state_t_18,state_t_19,state_t_20,state_t_21,state_t_22,state_t_23,state_t_24,state_t_25,state_t_26,state_t_27,state_t_28,state_t_29,state_t_30,state_t_31,state_t_32,state_t_33,state_t_34,state_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""mean""",215.612015,227.878262,237.309279,247.912285,256.227916,259.451296,255.271369,246.698109,236.980836,230.274286,225.054863,220.928353,217.108561,213.894105,210.537094,207.052169,202.847109,200.041338,199.271441,201.226921,203.712135,206.900332,210.434994,214.236472,218.053071,221.900889,225.667783,229.347179,232.912757,236.379712,239.742185,243.019442,246.204743,249.292946,252.269269,255.123318,…,-2.8680e-9,2.0688e-8,3.4469e-8,3.1557e-8,7.1815e-9,-6.3038e-8,-1.5727e-7,-2.4531e-7,-3.0017e-7,-3.0094e-7,-2.2172e-7,-7.6118e-8,1.1783e-7,3.2319e-7,4.9466e-7,6.1731e-7,6.3501e-7,5.3735e-7,3.3996e-7,8.3744e-8,-2.3336e-7,-6.8066e-7,-0.000001,-0.000001,-0.000001,-5.3330e-7,6.2734e-7,0.000001,6.5831e-7,158.311244,351.273302,2.7343e-9,2.9194e-8,61.803931,67.324685,33.451249,17.676368
"""std""",6.654266,8.653689,8.248617,6.786184,6.242818,8.249606,10.157319,10.112921,9.218096,8.81164,8.137254,7.43026,6.723265,6.321632,6.513009,7.6074,9.966565,11.792149,10.03546,8.244552,5.475349,4.073948,4.320332,5.478911,6.961327,8.309486,9.521203,10.517859,11.344936,12.00057,12.514359,12.893314,13.160995,13.32794,13.405421,13.407873,…,0.000008,0.000008,0.000007,0.000007,0.000007,0.000007,0.000008,0.000009,0.000009,0.00001,0.000011,0.000011,0.000012,0.000013,0.000014,0.000015,0.000016,0.000017,0.000018,0.00002,0.000022,0.000025,0.000028,0.000029,0.00003,0.000029,0.000028,0.000024,0.000034,246.661342,71.979277,7.3900e-9,8.1856e-8,110.255081,116.424189,46.397329,29.70672
"""q1_4""",211.795845,222.243162,231.759324,243.668977,252.799997,256.235228,251.635947,243.449935,234.437349,228.097416,223.197921,219.205218,215.187527,211.551624,207.162932,201.871982,194.857171,189.825833,190.678675,194.104774,199.497332,204.247093,208.499765,211.28707,213.509142,215.931538,218.53578,221.356252,224.305857,227.357786,230.46927,233.648802,236.847711,240.04982,243.221606,246.322576,…,-5.5398e-7,-5.2138e-7,-4.9227e-7,-4.6982e-7,-4.4817e-7,-4.3488e-7,-4.3559e-7,-4.4466e-7,-4.6692e-7,-5.0584e-7,-5.6217e-7,-6.3387e-7,-7.3155e-7,-8.5510e-7,-0.000001,-0.000001,-0.000001,-0.000002,-0.000002,-0.000003,-0.000003,-0.000004,-0.000005,-0.000006,-0.000007,-0.000008,-0.000007,-0.000006,-0.000009,0.0,311.199335,0.0,0.0,0.0,0.0,0.0,0.0
"""q2_4""",215.558519,228.043463,236.547149,247.060054,255.854526,260.29541,257.143055,248.743841,239.033878,232.473792,227.008668,222.471784,217.942912,213.953813,209.621341,205.616373,201.811222,199.379132,198.640697,200.417245,203.053164,206.646559,210.628659,215.049439,219.470938,223.699178,227.787865,231.76676,235.647794,239.425994,243.096914,246.65641,250.090561,253.374147,256.474984,259.427914,…,1.6653e-18,4.4409e-18,5.1810e-18,4.8110e-18,2.9606e-18,1.1102e-18,-2.3130e-18,-8.1416e-18,-1.2212e-17,-1.1842e-17,-3.7007e-18,3.7007e-19,5.9212e-18,5.9614e-16,1.5969e-14,2.1081e-13,4.4083e-13,1.6354e-13,8.4976e-14,4.3469e-15,5.4956e-17,-1.3878e-17,-3.

In [7]:
feat_stats_df.write_parquet(Path(data_dir, "feat_stats.parquet"))
label_stats_df.write_parquet(Path(data_dir, "label_stats.parquet"))
stats_df.write_parquet(Path(data_dir, "stats.parquet"))